# v3_1 层次 CTM（弱监督顶层 + 子层无监督）

目标：
- 使用 API 获取 4096 维嵌入（不降维），统一 NameEnglish + ShortDescription 作为文本；
- 顶层：使用 ZeroShotTM（弱监督）直接区分 Macro 与 Micro（2 类），通过种子词弱监督；
- 子层：在 Macro 子语料上选 K∈{2,3,4,5}，在 Micro 子语料上选 K∈{4,5,6,7}，用 CTM/CombinedTM 网格搜索并选择 3 与 5；
- 防过拟合：增大 dropout、weight decay、ReduceLROnPlateau（近似早停）、较小 hidden_sizes；
- 输出：主题词、代表文档（示例）、主题可视化（可选）、层次结构摘要，保存用于 v3_2。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U contextualized_topic_models

In [ ]:
# 环境安装（如需）
import sys, subprocess
def pip_install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

needed = [
    'contextualized-topic-models',
    'nltk', 'scikit-learn', 'gensim', 'openai', 'tqdm', 'pandas', 'numpy',
    'matplotlib', 'seaborn'
]
for p in needed:
    try:
        __import__(p.split('==')[0].replace('-', '_'))
    except Exception:
        pip_install(p)

In [ ]:
import os, json, time, re
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset
from openai import OpenAI
from contextualized_topic_models.models.ctm import CombinedTM, ZeroShotTM
nltk.download('stopwords')

# 配置路径（请根据自己的路径修改）
INPUT_JSON_PATH = '/content/drive/MyDrive/Colab Notebooks/验证3+5/gemini_merged_policy_data_with_labels_v2.json'
OUTPUT_DIR = '/content/drive/MyDrive/Colab Notebooks/验证3+5/v3_ctm_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)
EMBEDDED_JSON_PATH = os.path.join(OUTPUT_DIR, 'policies_with_embeddings_api_4096.json')
SPLIT_SUMMARY_PATH = os.path.join(OUTPUT_DIR, 'top_level_macro_micro_assignment.json')

# API 配置（从环境变量读取，避免明文）
api_key = os.environ.get('QWEN_API_KEY') or os.environ.get('CSTCLOUD_API_KEY')
BASE_URL = os.environ.get('QWEN_BASE_URL', 'https://uni-api.cstcloud.cn/v1')
EMBEDDING_MODEL = os.environ.get('QWEN_EMBED_MODEL', 'qwen3-embedding:8b')

# 随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_STATE)

In [ ]:
# 1) 生成 4096 维 API 嵌入，不做降维；统一使用 NameEnglish + ShortDescription

def load_policies(input_path):
    df = pd.read_json(input_path)
    # 统一文本：NameEnglish + ShortDescription；剔除过短
    df['text'] = (df.get('NameEnglish', '').fillna('') + '. ' + df.get('ShortDescription', '').fillna('')).astype(str)
    df = df[df['text'].str.len() > 10].reset_index(drop=True)
    return df

class Qwen3Embedding:
    def __init__(self, api_key, base_url, model='qwen3-embedding:8b', embedding_dim=4096, batch_size=100, sleep_s=0.2):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.sleep_s = sleep_s
    def encode(self, texts):
        embs = []
        for i in tqdm(range(0, len(texts), self.batch_size), desc='Embedding via API'):
            batch = texts[i:i+self.batch_size]
            resp = self.client.embeddings.create(model=self.model, input=batch)
            embs.extend([d.embedding for d in resp.data])
            time.sleep(self.sleep_s)
        return np.array(embs)

def ensure_embeddings(df, out_path):
    if os.path.exists(out_path):
        print('已存在嵌入文件，直接加载:', out_path)
        df2 = pd.read_json(out_path)
        return df2
    print('开始生成 4096 维嵌入...')
    encoder = Qwen3Embedding(api_key, BASE_URL, EMBEDDING_MODEL, 4096)
    embs = encoder.encode(df['text'].tolist())
    df = df.copy()
    df['embed'] = [e.tolist() for e in embs]
    df.to_json(out_path, force_ascii=False, orient='records', indent=2)
    print('嵌入已保存到:', out_path)
    return df

df_raw = load_policies(INPUT_JSON_PATH)
df_emb = ensure_embeddings(df_raw, EMBEDDED_JSON_PATH)
print('样本数:', len(df_emb))

In [ ]:
# 2) 文本预处理 & 词袋

def build_vectorizer(texts, min_df=5, max_df=0.9):
    base_sw = set(stopwords.words('english'))
    # 可按领域补充停用词（英文）
    domain_sw = set([
        'policy','policies','measure','measures','action','actions','law','laws','government','ministry','council',
        'support','development','research','innovation','technology','science','program','programs','programme',
        'country','countries','national','international','regional','local',
        'year','years','state','states'
    ])
    sw = list(base_sw | domain_sw)
    vectorizer = CountVectorizer(stop_words=sw, min_df=min_df, max_df=max_df)
    X = vectorizer.fit_transform(texts)
    vocab = vectorizer.get_feature_names_out()
    return vectorizer, X, vocab

vectorizer_all, bow_all, vocab_all = build_vectorizer(df_emb['text'].tolist())
bow_all = bow_all.toarray()
emb_all = np.array(df_emb['embed'].tolist(), dtype=np.float32)
print('BOW 维度:', bow_all.shape, 'Embed 维度:', emb_all.shape)

class CTMDataset(Dataset):
    # 简易 CTM 数据集，满足 CTM/ZeroShotTM 的输入
    def __init__(self, X_bow, X_contextual, idx2token):
        self.X_bow = torch.FloatTensor(X_bow)
        self.X_contextual = torch.FloatTensor(X_contextual)
        self.idx2token = idx2token  # 关键：用于 get_topic_lists 的词映射
    def __len__(self):
        return len(self.X_bow)
    def __getitem__(self, idx):
        return {'X_bow': self.X_bow[idx], 'X_contextual': self.X_contextual[idx]}

dataset_all = CTMDataset(bow_all, emb_all, vocab_all)

In [ ]:
# 3) 顶层：弱监督 ZeroShotTM 划分 Macro / Micro（2 类）
# 种子词设计（尽量概括宏观 vs 微观的语义差异）
macro_seeds = [
    'strategy','strategic','guideline','guidance','plan','planning','roadmap','blueprint','framework',
    'institutional','institution','governance','coordination','layout','agenda','vision','policy_document','reform'
]
micro_seeds = [
    'fund','funding','finance','subsidy','grant','tax','credit','loan','procurement','voucher','voucher',
    'talent','education','training','scholarship','recruitment','visa','immigration',
    'startup','incubator','accelerator','vc','venture','angel',
    'commercialization','transfer','licensing','patent','ip','intellectual','standard','regulation','compliance'
]
seed_topic_list = [macro_seeds, micro_seeds]

# 训练 ZeroShotTM（2 主题）。防过拟合：较大 dropout、小 hidden、ReduceLROnPlateau
zstm = ZeroShotTM(
    bow_size=len(vocab_all),
    contextual_size=emb_all.shape[1],  # 4096
    n_components=2,
    seed_topic_list=seed_topic_list,
    num_epochs=100,
    hidden_sizes=(64,),              # 较小隐层
    dropout=0.3,                     # 增强正则
    reduce_on_plateau=True,          # 类似早停的学习率退火
    optimizer_params={'lr': 2e-3},   # 稍低学习率
)
zstm.fit(dataset_all)

# 文档-话题分布，获取顶层预测
top_dist = zstm.get_doc_topic_distribution(dataset_all, n_samples=10)
top_pred = top_dist.argmax(axis=1)

# 将 0/1 对应到 Macro/Micro（根据种子主题顺序），并保存划分结果
label_map = {0: 'Macro', 1: 'Micro'}
df_emb_top = df_emb.copy()
df_emb_top['TopLevel'] = [label_map[i] for i in top_pred]
split_summary = df_emb_top['TopLevel'].value_counts().to_dict()
json.dump({'counts': split_summary}, open(SPLIT_SUMMARY_PATH, 'w'), ensure_ascii=False, indent=2)
print('顶层划分完成：', split_summary)

# 创建 mask 和子数据集，为后续子层训练准备
mask_macro = df_emb_top['TopLevel'] == 'Macro'
mask_micro = df_emb_top['TopLevel'] == 'Micro'

macro_df = df_emb_top[mask_macro].reset_index(drop=True)
micro_df = df_emb_top[mask_micro].reset_index(drop=True)

print(f'Macro 文档数: {len(macro_df)}, Micro 文档数: {len(micro_df)}')

In [ ]:
# 4) 子层：固定较大 K（总计 ~20）训练 CTM
# ===============================
# 思路：
# - 先统计 Macro/Micro 文档占比，据此把 TOT_K 按比例分配到两个子语料。
# - 再在各自子语料上用 CombinedTM 分别训练一次，得到较细的子层主题。
# - 这些“细主题”用于后续人工归并到 3+5。

TOT_K = 20  # 总主题数（可按需调整，比如 18/22/24）
K_MIN_MACRO, K_MAX_MACRO = 3, 12
K_MIN_MICRO, K_MAX_MICRO = 5, 16

n_macro = int(mask_macro.sum())
n_micro = int(mask_micro.sum())
n_total = n_macro + n_micro

# 按文档量比例分配 K
k_macro_prop = max(K_MIN_MACRO, min(K_MAX_MACRO, int(round(TOT_K * (n_macro / max(n_total,1))))))
k_micro_prop = TOT_K - k_macro_prop
# 保证 micro 在范围内；若微调造成越界，则回退
if k_micro_prop < K_MIN_MICRO:
    k_micro_prop = K_MIN_MICRO
    k_macro_prop = TOT_K - k_micro_prop
if k_micro_prop > K_MAX_MICRO:
    k_micro_prop = K_MAX_MICRO
    k_macro_prop = TOT_K - k_micro_prop
# 再次确保 macro 落在范围内（必要时对 TOT_K 做微调）
if k_macro_prop < K_MIN_MACRO:
    k_macro_prop = K_MIN_MACRO
if k_macro_prop > K_MAX_MACRO:
    k_macro_prop = K_MAX_MACRO

print(f'将总主题数 {TOT_K} 分配为：Macro={k_macro_prop}，Micro={k_micro_prop}')
print(f'文档数：Macro={n_macro}，Micro={n_micro}')

# 为每个子语料创建独立的词袋和数据集
macro_vec, macro_bow, macro_vocab = build_vectorizer(macro_df['text'].tolist())
micro_vec, micro_bow, micro_vocab = build_vectorizer(micro_df['text'].tolist())

macro_bow = macro_bow.toarray()
micro_bow = micro_bow.toarray()

macro_emb = np.array(macro_df['embed'].tolist(), dtype=np.float32)
micro_emb = np.array(micro_df['embed'].tolist(), dtype=np.float32)

macro_ds = CTMDataset(macro_bow, macro_emb, macro_vocab)
micro_ds = CTMDataset(micro_bow, micro_emb, micro_vocab)

print('子层数据集创建完成')
print(f'Macro BOW: {macro_bow.shape}, Embed: {macro_emb.shape}')
print(f'Micro BOW: {micro_bow.shape}, Embed: {micro_emb.shape}')

def train_ctm_fixed_k(dataset, vocab, k, max_epochs=200, lr=2e-3, dropout=0.35, hidden=(64,)):
    model = CombinedTM(
        bow_size=len(vocab), contextual_size=dataset.X_contextual.shape[1], n_components=k,
        num_epochs=max_epochs, hidden_sizes=hidden, dropout=dropout,
        reduce_on_plateau=True, optimizer_params={'lr': lr}
    )
    model.fit(dataset)
    return model

# 训练 Macro 子层
final_macro_model = train_ctm_fixed_k(macro_ds, macro_vocab, k_macro_prop)
final_macro_topics = final_macro_model.get_topic_lists(topk=15)
json.dump({'topics': final_macro_topics, 'k': int(k_macro_prop)}, open(os.path.join(OUTPUT_DIR, 'macro_topics.json'), 'w'), ensure_ascii=False, indent=2)

# 训练 Micro 子层
final_micro_model = train_ctm_fixed_k(micro_ds, micro_vocab, k_micro_prop)
final_micro_topics = final_micro_model.get_topic_lists(topk=15)
json.dump({'topics': final_micro_topics, 'k': int(k_micro_prop)}, open(os.path.join(OUTPUT_DIR, 'micro_topics.json'), 'w'), ensure_ascii=False, indent=2)

print('固定 K 的子层 CTM 训练完成。')

In [ ]:
# 5) 导出核心产物，供 v3_2 有监督/评估使用

# 顶层标签
df_emb_top.to_json(os.path.join(OUTPUT_DIR, 'df_with_top_level.json'), force_ascii=False, orient='records', indent=2)

# 为每个文档给出子层主题分配（可选）
def assign_topics(model, dataset):
    # 返回每个文档的主导主题 id
    d2t = model.get_doc_topic_distribution(dataset, n_samples=5)
    return d2t.argmax(axis=1)

macro_assign = assign_topics(final_macro_model, macro_ds)
micro_assign = assign_topics(final_micro_model, micro_ds)

macro_export = macro_df.copy()
macro_export['macro_topic_id'] = macro_assign
micro_export = micro_df.copy()
micro_export['micro_topic_id'] = micro_assign

macro_export.to_json(os.path.join(OUTPUT_DIR, 'macro_docs_simple.json'), force_ascii=False, orient='records', indent=2)
micro_export.to_json(os.path.join(OUTPUT_DIR, 'micro_docs_simple.json'), force_ascii=False, orient='records', indent=2)

# 层次结构摘要
hier_summary = {
    'top_level_counts': split_summary,
    'macro_best_k': int(k_macro_prop),
    'micro_best_k': int(k_micro_prop)
}
json.dump(hier_summary, open(os.path.join(OUTPUT_DIR, 'hierarchical_summary.json'), 'w'), ensure_ascii=False, indent=2)
print('导出完成。产物目录：', OUTPUT_DIR)

In [ ]:
# 新增单元：导出“主题目录”供人工判读 + 生成映射模板
# 放置位置：紧接在固定 K 训练单元之后（或导出单元之前）

def assign_topics_with_probs(model, dataset, topn=3):
    """为每篇文档分配主导主题，并保留前 topn 概率，便于人工核查"""
    d2t = model.get_doc_topic_distribution(dataset, n_samples=5)  # 平滑采样
    top_ids = d2t.argmax(axis=1)
    top_probs = d2t.max(axis=1)
    # 同时输出前 topn
    topn_ids = np.argsort(-d2t, axis=1)[:, :topn]
    topn_probs = np.take_along_axis(d2t, topn_ids, axis=1)
    return top_ids, top_probs, topn_ids, topn_probs

def build_topic_catalog(sub_df, model, topics, level_name, sample_per_topic=5):
    """构建可读的主题目录：每个主题的关键词、文档数、示例文本等"""
    top_ids, top_probs, topn_ids, topn_probs = assign_topics_with_probs(model,
                                                                        CTMDataset(
                                                                            np.array([x for x in sub_df['_bow']]),
                                                                            np.array([x for x in sub_df['_emb']]),
                                                                            None
                                                                        ),
                                                                        topn=3)

    # 把主导主题写回，便于后续导出
    sub_df = sub_df.copy()
    sub_df[f'{level_name}_topic_id'] = top_ids
    sub_df[f'{level_name}_topic_prob'] = top_probs

    # 统计每个主题的文档数
    from collections import Counter
    counts = Counter(top_ids)

    catalog = []
    for tid, words in enumerate(topics):
        # 示例文档（按该主题的主导概率排序取前 sample_per_topic 个）
        idxs = np.where(top_ids == tid)[0]
        if len(idxs) > 0:
            order = np.argsort(-top_probs[idxs])
            show = idxs[order][:sample_per_topic]
            samples = []
            for i in show:
                samples.append({
                    'NameEnglish': sub_df.iloc[i].get('NameEnglish', ''),
                    'ShortDescription': sub_df.iloc[i].get('ShortDescription', ''),
                    'score': float(top_probs[i])
                })
        else:
            samples = []
        catalog.append({
            'topic_id': int(tid),
            'doc_count': int(counts.get(tid, 0)),
            'keywords': words,
            'samples': samples
        })
    return sub_df, catalog

# 为构建 catalog 准备子语料里的 BOW/Emb（避免重复计算）
def augment_with_bow_emb(sub_df, vec):
    Xbow = vec.transform(sub_df['text']).toarray()
    Xemb = np.array(sub_df['embed'].tolist(), dtype=np.float32)
    sub_df = sub_df.copy()
    sub_df['_bow'] = list(Xbow)
    sub_df['_emb'] = list(Xemb)
    return sub_df

macro_df_aug = augment_with_bow_emb(macro_df, macro_vec)
micro_df_aug = augment_with_bow_emb(micro_df, micro_vec)

macro_df_cat, macro_catalog = build_topic_catalog(macro_df_aug, final_macro_model, final_macro_topics, 'macro')
micro_df_cat, micro_catalog = build_topic_catalog(micro_df_aug, final_micro_model, final_micro_topics, 'micro')

# 导出主题目录与文档-主题分配
json.dump({'level':'macro','k': int(len(macro_catalog)),'catalog': macro_catalog}, open(os.path.join(OUTPUT_DIR,'macro_topic_catalog.json'),'w'), ensure_ascii=False, indent=2)
json.dump({'level':'micro','k': int(len(micro_catalog)),'catalog': micro_catalog}, open(os.path.join(OUTPUT_DIR,'micro_topic_catalog.json'),'w'), ensure_ascii=False, indent=2)

macro_df_cat.drop(columns=['_bow','_emb']).to_json(os.path.join(OUTPUT_DIR,'macro_docs_with_topics.json'), orient='records', force_ascii=False, indent=2)
micro_df_cat.drop(columns=['_bow','_emb']).to_json(os.path.join(OUTPUT_DIR,'micro_docs_with_topics.json'), orient='records', force_ascii=False, indent=2)

# 生成人工映射模板（请人工填写 target_label 字段为 8 类中的一个）
EIGHT_CLASSES = [
    'Guideline_Strategy','Planning_Layout','Institutional_Arrangements',
    'Resource_Allocation_Policy','Innovation_Actor_Policy','Talent_Policy',
    'Commercialization_Policy','Environment_Shaping_Policy'
]

mapping_template = {
    'note': '请将每个子层的细主题映射到上述 8 类之一；未确定可暂留空字符串 ""',
    'allowed_target_labels': EIGHT_CLASSES,
    'macro': [{'topic_id': int(x['topic_id']), 'suggested_keywords': x['keywords'][:10], 'target_label': ''} for x in macro_catalog],
    'micro': [{'topic_id': int(x['topic_id']), 'suggested_keywords': x['keywords'][:10], 'target_label': ''} for x in micro_catalog]
}
json.dump(mapping_template, open(os.path.join(OUTPUT_DIR,'topic_mapping_template.json'),'w'), ensure_ascii=False, indent=2)
print('已导出：macro/micro 主题目录、文档-主题分配，以及 topic_mapping_template.json（请人工填写）。')

In [ ]:
# 新增单元：应用人工映射，将无监督细主题归并为 3+5，并做覆盖/一致性检查
# 放置位置：紧随“生成模板”单元之后

MAPPING_PATH = os.path.join(OUTPUT_DIR,'topic_mapping_template.json')  # 人工填写后的路径（可改名）

def apply_mapping_and_export(mapping_path):
    mapping = json.load(open(mapping_path,'r',encoding='utf-8'))
    allowed = set(mapping.get('allowed_target_labels', []))

    def build_map_dict(items):
        d = {}
        for it in items:
            tid = int(it['topic_id'])
            tgt = it.get('target_label','').strip()
            d[tid] = tgt
        return d

    macro_map = build_map_dict(mapping.get('macro',[]))
    micro_map = build_map_dict(mapping.get('micro',[]))

    # 载入文档-主题分配
    macro_docs = pd.read_json(os.path.join(OUTPUT_DIR,'macro_docs_with_topics.json'))
    micro_docs = pd.read_json(os.path.join(OUTPUT_DIR,'micro_docs_with_topics.json'))

    # 应用映射
    def map_row(row, level):
        tid = int(row[f'{level}_topic_id'])
        label = (macro_map if level=='macro' else micro_map).get(tid,'')
        return label

    macro_docs['MappedLabel'] = macro_docs.apply(lambda r: map_row(r,'macro'), axis=1)
    micro_docs['MappedLabel'] = micro_docs.apply(lambda r: map_row(r,'micro'), axis=1)

    # 覆盖情况
    macro_unmapped = (macro_docs['MappedLabel']=='').sum()
    micro_unmapped = (micro_docs['MappedLabel']=='').sum()
    print(f'Macro 未映射文档数：{macro_unmapped} / {len(macro_docs)}')
    print(f'Micro 未映射文档数：{micro_unmapped} / {len(micro_docs)}')

    # 合并
    merged = pd.concat([macro_docs, micro_docs], ignore_index=True)
    # 如果原始数据带有真值标签（ClassificationLabel），可以对齐评估一致性
    has_gt = 'ClassificationLabel' in merged.columns and merged['ClassificationLabel'].notna().any()
    if has_gt:
        from sklearn.metrics import confusion_matrix, classification_report

        valid = merged[merged['MappedLabel'].isin(allowed)]
        print(f'用于一致性检查的有效样本：{len(valid)}')

        y_true = valid['ClassificationLabel'].values
        y_pred = valid['MappedLabel'].values

        print('\n[无监督主题映射 vs 真值] Classification Report:')
        print(classification_report(y_true, y_pred, labels=sorted(allowed), zero_division=0))

        cm = confusion_matrix(y_true, y_pred, labels=sorted(allowed))
        plt.figure(figsize=(10,8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(allowed), yticklabels=sorted(allowed))
        plt.title('Unsupervised Topics (mapped) vs Ground Truth')
        plt.xlabel('Predicted (mapped)'); plt.ylabel('True')
        fn = os.path.join(OUTPUT_DIR, 'cm_unsup_mapped_vs_gt.png')
        plt.tight_layout(); plt.savefig(fn, dpi=150); plt.close()
        print('一致性混淆矩阵已保存：', fn)

    # 导出最终带映射标签的数据（可供后续监督训练或分析使用）
    merged.to_json(os.path.join(OUTPUT_DIR,'docs_with_unsup_mapped_label.json'), orient='records', force_ascii=False, indent=2)
    print('已导出：docs_with_unsup_mapped_label.json')

apply_mapping_and_export(MAPPING_PATH)